## Cross-Lingual Voice Clone Demo

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

c:\Users\user\Desktop\projects\openvoice_lib\OpenVoice\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In [2]:
ckpt_converter = 'checkpoints/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

tone_color_converter = ToneColorConverter(f'checkpoints\config (1).json', device=device)
tone_color_converter.load_ckpt(f'checkpoints\checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

c:\Users\user\Desktop\projects\openvoice_lib\OpenVoice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'checkpoints\checkpoint.pth'
missing/unexpected keys: [] []


In this demo, we will use OpenAI TTS as the base speaker to produce multi-lingual speech audio. The users can flexibly change the base speaker according to their own needs. Please create a file named `.env` and place OpenAI key as `OPENAI_API_KEY=xxx`. We have also provided a Chinese base speaker model (see `demo_part1.ipynb`).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

# Please create a file named .env and place your
# OpenAI key as OPENAI_API_KEY=xxx
load_dotenv() 

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input="This audio will be used to extract the base speaker tone color embedding. " + \
        "Typically a very short audio should be sufficient, but increasing the audio " + \
        "length will also improve the output audio quality."
)

response.stream_to_file(f"{output_dir}/openai_source_output.mp3")

### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average for multiple sentences with multiple emotions
of the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

In [3]:
reference_speaker = "resources/outputstmpgcva2e8d.wav" # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v2


In [7]:
print(target_se.shape, type(target_se))
print(target_se)

torch.Size([1, 256, 1]) <class 'torch.Tensor'>
tensor([[[ 6.5086e-01],
         [ 3.3810e-01],
         [-7.4469e-01],
         [ 5.9212e-02],
         [-1.1121e-01],
         [ 5.8433e-01],
         [ 3.5062e-01],
         [ 6.0381e-01],
         [-1.6316e-01],
         [ 4.8495e-01],
         [-4.0005e-01],
         [ 3.0792e-01],
         [ 2.0854e-02],
         [-5.5919e-01],
         [-6.3440e-01],
         [ 5.6704e-02],
         [ 3.1845e-03],
         [ 2.6211e-01],
         [ 2.4915e-01],
         [-9.0036e-03],
         [ 2.3058e-01],
         [-1.3958e+00],
         [ 8.5750e-01],
         [-1.0627e+00],
         [-1.2803e+00],
         [ 3.6978e-01],
         [-4.4672e-01],
         [-2.6727e-01],
         [ 4.4269e-01],
         [-2.6961e-01],
         [-9.4873e-01],
         [-6.2493e-01],
         [-3.3114e-01],
         [ 1.5956e-01],
         [ 8.6794e-03],
         [ 1.3661e-01],
         [ 2.3262e-01],
         [ 5.1134e-01],
         [-3.4378e-01],
         [-2.6038

### Inference

In [ ]:
# Run the base speaker tts
text = [
    "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.",
    "MyShell es una plataforma descentralizada y completa para descubrir, crear y apostar por aplicaciones nativas de IA.",
    "MyShell est une plateforme décentralisée et complète pour découvrir, créer et miser sur des applications natives d'IA.",
    "MyShell ist eine dezentralisierte und umfassende Plattform zum Entdecken, Erstellen und Staken von KI-nativen Apps.",
    "MyShell è una piattaforma decentralizzata e completa per scoprire, creare e scommettere su app native di intelligenza artificiale.",
    "MyShellは、AIネイティブアプリの発見、作成、およびステーキングのための分散型かつ包括的なプラットフォームです。",
    "MyShell — это децентрализованная и всеобъемлющая платформа для обнаружения, создания и стейкинга AI-ориентированных приложений.",
    "MyShell هي منصة لامركزية وشاملة لاكتشاف وإنشاء ورهان تطبيقات الذكاء الاصطناعي الأصلية.",
    "MyShell是一个去中心化且全面的平台，用于发现、创建和投资AI原生应用程序。",
    "MyShell एक विकेंद्रीकृत और व्यापक मंच है, जो AI-मूल ऐप्स की खोज, सृजन और स्टेकिंग के लिए है।",
    "MyShell é uma plataforma descentralizada e abrangente para descobrir, criar e apostar em aplicativos nativos de IA."
]
src_path = f'{output_dir}/tmp.wav'

for i, t in enumerate(text):

    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=t,
    )

    response.stream_to_file(src_path)

    save_path = f'{output_dir}/output_crosslingual_{i}.wav'

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)